In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.integrate as inte
from numpy import matmul as mm
from mpl_toolkits import mplot3d

### Reusable Functions

In [3]:
def quad_pos(pos,rot,L,H=.05):
    tmp = np.hstack((rot,pos))
    wHb = np.vstack((tmp,np.array([0,0,0,1])))
    quadBodyFrame = np.array([[L,0,0,1],
                             [0,L,0,1],
                             [-L,0,0,1],
                             [0,-L,0,1],
                             [0,0,0,1],
                             [0,0,H,1]]).T
    quadWorldFrame = mm(wHb,quadBodyFrame)
    quad = quadWorldFrame[:3,:]
    return quad
def QuatToRot(q):
    q = q/np.sqrt(np.sum(q**2))
    qahat = np.zeros((3,3))
    qahat[0,1],qahat[0,2] = -q[3],q[2]
    qahat[1,2],qahat[1,0] = -q[1],q[3]
    qahat[2,0],qahat[2,1] = -q[2],q[1]
    R = np.eye(3)+2*mm(qahat,qahat)+2*q[0]*qahat
    return R

### Modified Functions

In [2]:
def sys_params():
    params = {}
    params['gravity'],params['mass'],params['arm_length']=9.81,.18,.086
    params['Ixx'] = .00025
    params['minF'] = 0
    params['maxF'] = 2*params['mass']*params['gravity']
    return params
def simStateToQuadState(sim_state):
    quad_state = np.zeros((13,1))
    quad_state[0],quad_state[3],quad_state[12] = 0,0,0
    quad_state[8:11] = 0
    quad_state[1] = sim_state[0]
    quad_state[2] = sim_state[1]
    quad_state[4] = sim_state[3]
    quad_state[5] = sim_state[4]
    quad_state[11] = sim_state[5]
    quad_state[6] = np.cos(-sim_state[2]/2)
    quad_state[7] = np.sin(-sim_state[2]/2)
    return quad_state
def sys_eom(t,s,controlhandle,trajhandle,params):
    current_state = {}
    current_state['pos'] = s[:2]
    current_state['rot'] = s[2]
    current_state['vel'] = s[3:5]
    current_state['omega'] = s[5]
    desired_state = trajhandle(t,current_state)
    F,M = controlhandle(t,current_state,desired_state,params)
    u1 = .5*(F-M/params['arm_length'])
    u2 = .5*(F+M/params['arm_length'])
    u1_clamped = np.clip(u1,params['minF']/2,params['maxF']/2)
    u2_clamped = np.clip(u2,params['minF']/2,params['maxF']/2)
    F_clamped = u1_clamped + u2_clamped
    M_clamped = (u2_clamped-u1_clamped)*params['arm_length']
    sdot = np.array([[s[3]],[s[4]],[s[5]],
                    [-F_clamped*np.sin(s[2])/params['mass']],
                    [F_clamped*np.cos(s[2])/params['mass']-params['gravity']],
                    [M_clamped/params['Ixx']]])
    return sdot

### QuadPlot Class

In [ ]:
class QuadPlot():
    def __init__(self,qn,state,wingspan,height,color,max_iter,h_3d=None):
        # public
        self.k,self.time = 0,0
        self.qn = qn
        self.state = state
        self.wingspan = wingspan
        self.color = color
        self.height = height
        self.rot = QuatToRot(self.state[6:10])
        self.motor = quad_pos(self.state[:3],self.rot,self.wingspan,self.height)
        self.max_iter = max_iter
        self.state_hist = np.zeros((6,max_iter))
        self.time_hist = np.zeros((1,max_iter))
        
        # private
        self.__text_dist = self.wingspan/3
        if h_3d is None:
            h_3d = plt.gca()
        self.__h_3d = h_3d
        self.__h_pos_hist = h_3d.plot3D(self.state[0],
                                        self.state[1],self.state[2],'r.')
        self.__h_m13 = h_3d.plot3D(self.motor[0,[0,2]],
                                  self.motor[1,[0,2]],
                                  self.motor[2,[0,2]],'-ko',
                                  markerfacecolor=self.color,
                                  markersize=5)
        self.__h_m24 = h_3d.plot3D(self.motor[0,[1,3]],
                                  self.motor[1,[1,3]],
                                  self.motor[2,[1,3]],'-ko',
                                  markerfacecolor=self.color,
                                  markersize=5)
        self.__h_qz = h_3d.plot3D(self.motor[0,[4,5]],
                                  self.motor[1,[4,5]],
                                  self.motor[2,[4,5]],
                                  color=self.color,
                                  linewidth=2)
#         self.__h_qn = 
    def UpdateQuadState(self,state,time):
        self.state = state
        self.time = time
        self.rot = QuatToRot(state[6:10]).T
    def UpdateQuadHist(self):
        self.k += 1
        self.time_hist[0,self.k] = self.time
        self.state_hist[:,self.k] = self.state[:6].flatten()
    def UpdateMotorPos(self):
        self.motor = quad_pos(self.state[:3],self.rot,self.wingspan,self.height)
    def TruncateHist(self):
        self.time_hist = self.time_hist[:self.k]
        self.state_hist = self.state_hist[:,:self.k]
    def UpdateQuadPlot(self,state,time):
        self.UpdateQuadState(state,time)
        self.UpdateQuadHist()
        self.UpdateMotorPos()
        h_3d = plt.gca()
        self.__h_pos_hist = h_3d.plot3D(self.state_hist[0,:self.k],
                                        self.state_hist[1,:self.k],
                                        self.state_hist[2,:self.k],'r.')
        self.__h_m13 = h_3d.plot3D(self.motor[0,[0,2]],
                                  self.motor[1,[0,2]],
                                  self.motor[2,[0,2]],'-ko',
                                  markerfacecolor=self.color,
                                  markersize=5)
        self.__h_m24 = h_3d.plot3D(self.motor[0,[1,3]],
                                  self.motor[1,[1,3]],
                                  self.motor[2,[1,3]],'-ko',
                                  markerfacecolor=self.color,
                                  markersize=5)
        self.__h_qz = h_3d.plot3D(self.motor[0,[4,5]],
                                  self.motor[1,[4,5]],
                                  self.motor[2,[4,5]],
                                  color=self.color,
                                  linewidth=2)
        fig = plt.gcf()
        fig.canvas.draw()

### Trajectory Library

In [4]:
def traj_line(t):
    return desired_state
def traj_diamond(t):
    return desired_state
def traj_sine(t):
    return desired_state
def traj_step(t):
    return desired_state